# 알고리즘

1. 기사 전처리 
    - 빅카인즈 데이터  --> 키워드 사용
    - 어린이 동아 --> 토크나이저 (명사)
2. tf-idf, countVectorizer 
    - 빅카인즈 데이터 기준 -> l2 norm, normalize
    - 어린이 동아 transform 
3. consine similarity 계산


In [2]:
import glob
import os

# 특정 폴더 경로 지정
kids_path = '/home/newsGuardian/Dataset/1'
bigs_path =  '/home/newsGuardian/Dataset/0'

# 해당 폴더에서 모든 .txt 파일 찾기

kids_files = glob.glob(os.path.join(kids_path, '*.txt'))
bigs_files = glob.glob(os.path.join(bigs_path, '*.txt'))

kids_files.sort()
bigs_files.sort()
print(kids_files)


['/home/newsGuardian/Dataset/1/10000001.txt', '/home/newsGuardian/Dataset/1/10000002.txt', '/home/newsGuardian/Dataset/1/10000003.txt', '/home/newsGuardian/Dataset/1/10000004.txt', '/home/newsGuardian/Dataset/1/10000005.txt', '/home/newsGuardian/Dataset/1/10000006.txt', '/home/newsGuardian/Dataset/1/10000007.txt', '/home/newsGuardian/Dataset/1/10000008.txt', '/home/newsGuardian/Dataset/1/10000009.txt', '/home/newsGuardian/Dataset/1/10000010.txt', '/home/newsGuardian/Dataset/1/10000011.txt', '/home/newsGuardian/Dataset/1/10000012.txt', '/home/newsGuardian/Dataset/1/10000013.txt', '/home/newsGuardian/Dataset/1/10000014.txt', '/home/newsGuardian/Dataset/1/10000015.txt', '/home/newsGuardian/Dataset/1/10000016.txt', '/home/newsGuardian/Dataset/1/10000017.txt', '/home/newsGuardian/Dataset/1/10000018.txt', '/home/newsGuardian/Dataset/1/10000019.txt', '/home/newsGuardian/Dataset/1/10000020.txt', '/home/newsGuardian/Dataset/1/10000021.txt', '/home/newsGuardian/Dataset/1/10000022.txt', '/home/ne

In [3]:
print(bigs_files[:10])

['/home/newsGuardian/Dataset/0/00000001.txt', '/home/newsGuardian/Dataset/0/00000002.txt', '/home/newsGuardian/Dataset/0/00000003.txt', '/home/newsGuardian/Dataset/0/00000004.txt', '/home/newsGuardian/Dataset/0/00000005.txt', '/home/newsGuardian/Dataset/0/00000006.txt', '/home/newsGuardian/Dataset/0/00000007.txt', '/home/newsGuardian/Dataset/0/00000008.txt', '/home/newsGuardian/Dataset/0/00000009.txt', '/home/newsGuardian/Dataset/0/00000010.txt']


In [4]:
# make data frame
# 데이터를 저장할 빈 리스트
from tqdm import tqdm
import pandas as pd

data_kids = []
data_bigs = []

# 각 파일을 순회하며 데이터 읽기
for file in tqdm(kids_files):
    with open(file, 'r') as f:
        lines = f.readlines()
        title = lines[0].strip()  # 첫 번째 줄 (제목)
        content = ' '.join(lines[1:]).strip()  # 나머지 줄 (내용)
        data_kids.append([title, content])

for file in tqdm(bigs_files):
    with open(file, 'r') as f:
        lines = f.readlines()
        title = lines[0].strip()  # 첫 번째 줄 (제목)
        content = ' '.join(lines[1:]).strip()  # 나머지 줄 (내용)
        data_bigs.append([title, content])



# 데이터 프레임 생성
df_kids = pd.DataFrame(data_kids, columns=['Title', 'Content'])
df_bigs = pd.DataFrame(data_bigs, columns=['Title', 'Content'])




100%|██████████| 35428/35428 [00:05<00:00, 6278.44it/s] 


## EDA

In [9]:
# 데이터 전처리 -- 토크나이저
# Konlpy에서 Okt를 불러옵니다.
from konlpy.tag import Okt

# Okt Class의 생성자를 이용하여 분석기를 생성 
okt = Okt()

def tokenizer(raw, pos=["Noun","Alpha", "Number"], stopword=[]): 
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   
            stem=True    
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [31]:
### data 의 nan 값 확인 

# 'contents' 열의 값이 Null인 행을 필터링
null_contents = df_bigs[df_bigs['Content'].isnull()]

# 해당 행의 'title' 열 값을 확인
titles_with_null_contents = null_contents['Title']

# 결과 출력
print(titles_with_null_contents)


83                                    2023년 11월 4일 글로벌 NOW
185                                  2023년 10월 28일 글로벌 NOW
548                   법무부, '로톡 이용 변호사 징계' 3차 심의 돌입...결론 주목
575                      청소년 대상 불법 도박사이트 특별단속...개인방송 광고 주시
598                                   2023년 9월 23일 글로벌 NOW
                               ...                        
29403                           한미일 국방장관 통화...정상회의 후속조치 논의
29751    [뉴스킹] 김성주 "대통령실 홍보영상 57개중 39개가 오염수 관련...日정부가 돈...
30550                [미리보는오늘] 윤 대통령 오늘 美 출국...이재명 대표 검찰 출석
30561                        尹, 오늘 장례절차 마치고 '한미일 정상회의' 미국행
30804                          김윤덕 "스카우트 대원에 사과...국정조사 필요"
Name: Title, Length: 185, dtype: object


In [36]:
dropna_bigs = df_bigs.dropna(axis=0)
dropna_bigs.isnull().sum()

Title      0
Content    0
dtype: int64

In [37]:
dropna_bigs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35243 entries, 0 to 35427
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    35243 non-null  object
 1   Content  35243 non-null  object
dtypes: object(2)
memory usage: 826.0+ KB


In [38]:
dropna_kids = df_kids.dropna(axis=0)
dropna_kids.isnull().sum()

Title      0
Content    0
dtype: int64

In [7]:
# dropna_kids.to_csv('./dropna_kids.csv', index=False)
# dropna_bigs.to_csv('./dropna_bigs.csv', index=False)

dropna_bigs = pd.read_csv('./dropna_bigs.csv')
dropna_kids = pd.read_csv('./df_kids.csv')

In [11]:
dropna_kids.isnull().sum()

Title       0
Content    25
dtype: int64

In [12]:
dropna_bigs.isnull().sum()

Title      0
Content    0
dtype: int64

In [13]:
dropna_bigs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35243 entries, 0 to 35242
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    35243 non-null  object
 1   Content  35243 non-null  object
dtypes: object(2)
memory usage: 550.8+ KB


In [14]:
# 모든 데이터를 문자열로 변환
dropna_bigs['Content'] = dropna_bigs['Content'].astype(str)
dropna_kids['Content'] = dropna_kids['Content'].astype(str)


In [15]:
# DataFrame의 각 내용을 토큰화


dropna_kids['tokenized_content'] = dropna_kids['Content'].apply(lambda x: ' '.join(tokenizer(x)))
dropna_bigs['tokenized_content'] = dropna_bigs['Content'].apply(lambda x : ' '.join(tokenizer(x)))

In [17]:
dropna_bigs.to_csv('./token_bigs.csv', index=False)
dropna_kids.to_csv('./token_kids.csv', index=False)

## vectorize

In [18]:
# bigs kinds news fit transform

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# CountVectorizer 초기화 및 적용
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(dropna_bigs['tokenized_content'])

# TfidfTransformer 초기화 및 적용
tfidf_transformer = TfidfTransformer(norm='l2')
X_tfidf = tfidf_transformer.fit_transform(X_counts)


In [19]:
# kids news transform

Y_conunts = count_vect.transform(dropna_kids['tokenized_content'])
Y_tfidf = tfidf_transformer.transform(Y_conunts)

`consine_similarity` : 사이킷런에서 제공하는 API (대상, 전체) 로 사용한다


## 유사도 판단

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 유사도가 0.9 이상 0.999 이하인 기사를 저장할 리스트 초기화
similar_articles = []

# Y_tfidf의 각 벡터(어린이용 기사)에 대해 반복
for i in range(Y_tfidf.shape[0]):
    # i번째 어린이용 기사와 모든 X_tfidf 기사 간의 유사도 계산
    cos_similarities = cosine_similarity(Y_tfidf[i], X_tfidf)

    # 유사도 필터링 (0.9 이상 0.999 이하)
    filtered_indices = np.where((cos_similarities[0] >= 0.7) & (cos_similarities[0] <= 0.999))[0]

    # 필터링된 인덱스와 해당 유사도 저장
    for idx in filtered_indices:
        similar_articles.append((i, idx, cos_similarities[0][idx]))

# 결과 출력
for (child_idx, news_idx, similarity) in similar_articles:
    print(f"어린이용 기사 인덱스: {child_idx}, 유사한 뉴스 기사 인덱스: {news_idx}, 유사도: {similarity}")


어린이용 기사 인덱스: 81, 유사한 뉴스 기사 인덱스: 10967, 유사도: 0.835016366714703
어린이용 기사 인덱스: 81, 유사한 뉴스 기사 인덱스: 10968, 유사도: 0.8771342140514867
어린이용 기사 인덱스: 83, 유사한 뉴스 기사 인덱스: 11744, 유사도: 0.7182288856831995
어린이용 기사 인덱스: 84, 유사한 뉴스 기사 인덱스: 53, 유사도: 0.7504144689210519
어린이용 기사 인덱스: 84, 유사한 뉴스 기사 인덱스: 5098, 유사도: 0.7504144689210519
어린이용 기사 인덱스: 88, 유사한 뉴스 기사 인덱스: 870, 유사도: 0.7393099167215494
어린이용 기사 인덱스: 90, 유사한 뉴스 기사 인덱스: 5138, 유사도: 0.710596747869087
어린이용 기사 인덱스: 109, 유사한 뉴스 기사 인덱스: 22174, 유사도: 0.7095683371520543
어린이용 기사 인덱스: 111, 유사한 뉴스 기사 인덱스: 11403, 유사도: 0.7192636387176258
어린이용 기사 인덱스: 111, 유사한 뉴스 기사 인덱스: 15184, 유사도: 0.7193738348642758
어린이용 기사 인덱스: 115, 유사한 뉴스 기사 인덱스: 11189, 유사도: 0.7910974356155632
어린이용 기사 인덱스: 117, 유사한 뉴스 기사 인덱스: 22186, 유사도: 0.7954287847425239
어린이용 기사 인덱스: 121, 유사한 뉴스 기사 인덱스: 5558, 유사도: 0.7073536515686227
어린이용 기사 인덱스: 132, 유사한 뉴스 기사 인덱스: 9897, 유사도: 0.8516400243241904
어린이용 기사 인덱스: 147, 유사한 뉴스 기사 인덱스: 6071, 유사도: 0.765444288752064
어린이용 기사 인덱스: 149, 유사한 뉴스 기사 인덱스: 15883, 유사도: 0.7057405204386

In [21]:
dropna_kids.iloc[316]['Content']

'북극곰들의 사냥지인 바다 얼음이 사라지면서 이들은 빙상 표면에서 먹이를 먹고 산다고 알려진다. CNN 홈페이지 캡처 \u200b연구진은 허드슨만 서부 지역 새끼 북극곰의 생존율이 현재 49%까지 떨어질 것으로 추산했다 북극곰들이 먹이를 먹지 못하는 기간이 40년 전보다 11배 이상 늘었다는 연구 결과가 나왔어요. 미국 워싱턴대와 와이오밍대, 북극곰 보호단체 ‘폴라베어인터내셔널’의 공동 연구진은 온실가스 배출량이 북극곰의 주요 서식지에 미치는 영향을 계산한 논문을 최근 과학저널 ‘사이언스’에 발표했어요. 특정 온실가스 배출량과 북극곰 생존율 사이의 상관관계를 구체적인 수치로 나타낸 연구결과는 이번이 처음이에요. 바다 얼음 위에서 먹잇감을 사냥하는 북극곰들은 지구온난화로 인해 사냥터가 줄어들면서 치열해진 먹이경쟁에 굶는 날이 많아지고 있어요. 이에 연구진은 북극곰 주요 서식지 15곳에서 북극곰들이 먹이를 찾지 못해 굶은 기간(단식 기간)과 온실가스 배출량을 분석했어요. 그리곤 새끼 북극곰이 독립할 때까지 살아남는 비율과의 상관관계를 계산했지요. 그 결과 축치 해(러시아 축치 자치구와 미국 알래스카 주 사이에 위치한 바다)에 서식하는 북극곰들의 단식 기간이 1979년 12일에서 2020년 137일로 11배 이상 늘어난 것으로 나타났어요. 보퍼트 해(미국 알래스카 주 북쪽 연안과 캐나다의 밴쿠버 섬 부근에 있는 바다) 북극곰의 단식 기간은 같은 기간 50.7일에서 125일로 두 배 이상 늘었지요. 이와 함께 연구진이 온실가스 배출량과 새끼 북극곰 생존율을 분석한 결과 가장 유명한 북극곰 서식지 중 하나인 캐나다 북동부 허드슨만 서부 지역의 경우 1980년대에 70% 수준이던 생존율이 현재 49%로 떨어질 것으로 추산(미루어 셈함)됩니다. 연구진은 이 같은 결과가 수년간 관찰돼 온 북극곰 개체 수 감소 추정치와 일치한다면서 이를 고려하면 현재 연간 새끼 북극곰의 생존율은 32% 정도일 수 있다고 설명했어요. 한편 연구진은 이번 연구 결과가 멸종위기종인 북극곰을 보호하

In [22]:
dropna_bigs.iloc[9338]['Content']

'사진 출처=로이터/연합뉴스북극곰이 먹이를 찾지 못해 굶는 날이 40년새 최대 11배나 증가한 것으로 나타났다.미국 워싱턴대·와이오밍대와 북극곰 보호단체 \'폴라베어인터내셔널\'의 공동 연구진은 온실가스 배출량이 북극곰 주요 서식지에 미치는 영향을 계산한 논문을 지난달 31일(현지 시각) 과학저널 \'사이언스\'에 발표했다.연구진은 북극곰 주요 서식지 15곳에서 온실가스 배출량과 사냥터인 바다 얼음(해빙)이 없어 북극곰이 먹이를 찾지 못해 굶는 기간(단식기간)을 분석하고, 새끼 북극곰이 독립할 때까지 살아남는 비율과의 상관관계를 정량화했다.이들은 축치해에 서식하는 북극곰들은 1979년 12일이었던 단식 기간이 2020년 137일로 11배 이상으로 길어졌고, 보퍼트해 북극곰의 단식 기간은 같은 기간 50.7일에서 125일로 두 배 이상 길어졌다고 전했다.또 온실가스가 대기 중으로 14Gt(기가톤) 방출될 때마다 축치해 북극곰의 단식 기간이 하루씩 늘어난다고 분석했다.이와 함께 북극곰 단식 기간이 연 117일을 넘는 해가 5년 사이 3년 연속 이어지는 경우를 \'신규개체 충격 한계선\'으로 설정하고, 이때를 전후로 온실가스 배출량과 새끼 북극곰 생존율을 계산했다.허드슨만 서부 지역의 경우 1994년에 이 한계선을 넘었다. 1994년∼2020년 사이 온실가스 배출량을 고려하면 1980년대에 70% 수준이던 새끼 북극곰의 생존율이 현재는 49%로 떨어졌을 것으로 추산된다.연구진은 이러한 계산 결과가 최근 수년간 관찰된 북극곰 개체수 감소 추정치와 일치한다면서 "2021년 수치로는 이 지역 북극곰 개체수가 1980년대 후반 이후 거의 절반으로 줄어든 것으로 나타났는데 이를 고려하면 현재 연간 새끼 북극곰의 생존율은 32% 정도일 수 있다"고 설명했다.또한 미국에 있는 발전소 수백 곳에서 30년 동안 배출하는 온실가스 60Gt이 남보퍼트해의 북극곰 생존율을 최대 4%p 낮출 수 있다고 덧붙였다.미국은 2008년 북극곰을 멸종위기종으로 지정했지만 북극곰 보호를 위한 화석연

In [23]:
print(dropna_kids.iloc[1800]['Content'])

바나나는 과일 스무디의 단골 재료예요. 부드럽고 달콤해서 딸기나 블루베리 등 베리류의 새콤함을 잡아주죠. 그런데 앞으로 섞지 말고 따로 먹어야겠어요. 8월 24일(현지 시각), 국제 학술지 '푸드 앤 펑션'에 바나나를 다른 과일 스무디에 섞어 넣으면 건강에 좋은 항산화 성분인 '플라바놀'이 훨씬 적게 흡수된다는 연구 결과가 실렸어요. ▲사과 ▲배 ▲딸기 ▲블루베리 ▲블랙베리 ▲포도 ▲코코아 등으로 섭취할 수 있는 플라바놀은 심장 건강과 인지 건강에 좋은 성분이에요.미국 캘리포니아대 데이비스 캠퍼스 내과, 사우디아라비아 킹사우드대, 영국 레딩대 등 국제 공동 연구진은 바나나가 들어간 스무디가 흔한 만큼 건강한 과일 조합을 찾기 위해 실험을 진행했어요. 건강한 남성으로 구성된 참가자를 두 그룹으로 나눠 한 그룹에는 바나나가 들어간 스무디를, 다른 그룹에는 베리류를 섞어 만든 스무디를 마시도록 했죠. 또 이들에게 플라바놀 캡슐을 먹게 한 뒤, 이들의 혈액과 소변 검체를 분석해 체내 플라바놀 수치를 측정했답니다.그 결과 바나나가 들어간 스무디를 마신 그룹은 대조군에 비해 체내 플라바놀 수치가 84% 낮았어요. 바나나에 많이 들어 있는 폴리페놀산화효소 때문이었어요. 플라바놀은 폴리페놀산화효소와 만나면 체내 흡수가 어려워지는 성질을 갖고 있거든요. 연구진은 "바나나 한 개를 추가할 경우 플라바놀 흡수율이 얼마나 빨리 떨어지는지 확인하고 깜짝 놀랐다"며 "플라바놀을 섭취하려는 사람들은 파인애플, 오렌지, 망고 등 폴리페놀산화효소가 적게 든 과일과 함께 먹어야 한다"고 전했답니다.


In [24]:
print(dropna_bigs.iloc[4160]['Content'])

바나나와 다른 과일을 섞어 스무디를 만들어 마시면 건강 효과가 떨어질 수 있다는 연구 결과가 나왔습니다.24일(현지시간) 미국 캘리포니아대 데이비스캠퍼스 연구팀에 따르면, 바나나를 다른 과일 스무디에 넣었을 때 항산화 성분인 '플라바놀'이 훨씬 적게 흡수됐습니다.플라바놀은 심장 기능과 기억력 등 인지 능력 향상에 좋은 생리활성화합물로, 사과, 배, 블루베리, 포도, 코코아 등을 통해 섭취할 수 있습니다.최근에는 플라바놀이 풍부한 음식을 섭취한 노인들의 기억 기능이 개선됐다는 연구 결과가 나오기도 했습니다.연구팀은 피실험군을 두개로 나눠 한 그룹은 바나나를 섞은 스무디를, 다른 그룹은 베리류로 만든 스무디를 제공한 뒤 플라바놀 캡슐을 섭취하게 했습니다.이후 이들의 혈액과 소변 검체를 분석해 체내 플라바놀 수치를 측정한 결과, 바나나 스무디를 마신 그룹의 수치가 대조군에 비해 84%나 낮았습니다.플라바놀은 폴리페놀산화효소(PPO)와 만나면 체내 흡수가 어려워지는데, 바나나에 이 성분이 많기 때문이라는 설명입니다.하비에르 오타비아니 연구원은 "바나나 한 개를 추가할 경우 플라바놀 흡수율이 얼마나 빨리 떨어지는지 확인해 깜짝 놀랐다"며 "이는 음식의 조합이 식이화합물 흡수에 어떤 영향을 미치는지 보여준다"고 말했습니다.이어 "플라바놀을 섭취하려는 사람들은 파인애플, 오렌지, 망고 등 폴리페놀산화효소가 낮은 과일과 함께 먹어야 한다"고 당부했습니다.제작 : 이선YTN 서미량 (tjalfid@ytn.co.kr)[저작권자(c) YTN 무단전재, 재배포 및 AI 데이터 활용 금지]


In [28]:
#어린이용 기사 인덱스: 2844, 유사한 뉴스 기사 인덱스: 31687, 유사도: 0.7217307024446616

print(dropna_kids.iloc[2844]['Content'])
print(dropna_bigs.iloc[31687]['Content'])

"그 누가 아무리 자기네 땅이라고 우겨도 독도는 우리 땅! 우리 땅!"노랫말에도 나오듯, 독도는 대한민국의 고유 영토입니다. 옛날부터 울릉도 주민들이 독도 앞바다에서 활발한 어업 활동을 벌이기도 했죠. 화창한 날이면 울릉도에서 맨눈으로 독도를 확인할 수도 있는데요.세종대왕이 우리나라 각 도의 정보를 모아 만든 '세종실록 지리지'에도 날이 좋으면 울릉도에서 독도가 보였다는 내용이 적혀 있답니다. 오늘날 독도에 대한 일본 정부의 부당한 주장이 이어지고 있지만, 우리나라뿐만 아니라 일본의 고서에도 독도는 한국 영토라는 사실이 정확히 기입돼 있답니다.1695년 당시 일본 지도층인 에도 막부의 질문에 답한 '돗토리번답변서'를 보면 '울릉도가 일본 돗토리현에 속하는 섬인가?'라는 질문에 '다케시마(울릉도)와 마쓰시마(독도)는 이나바와 호키(지금의 돗토리현)에 속하는 섬이 아닙니다'라고 답변한 기록이 남아 있습니다. 1877년 일본의 최고 국가기관인 태정관에서 기록한 '태정관지령'에는 '다케시마(울릉도) 외 일도(독도)의 건은 본방(本邦=일본)과는 관계가 없음을 명심할 것'이라는 내용과 독도는 일본 땅이 아님을 표시한 '기죽도약도'가 그려져 있답니다.1880년 일본 내무성 지리국이 제작한 '대일본국전도'에는 독도가 포함돼 있지 않죠. 이 밖에도 독도는 일본 땅이 아님을 인정하는 문서와 지도들이 기록으로 남아 있는데요. 모두 역사적, 지리적, 국제법적으로 독도가 명백히 우리나라 영토라고 적혀 있답니다.독도를 지키기 위한 노력은 오늘날까지 계속되고 있어요. 독도의 주권 강화를 위해 한국해양과학기술원의 울릉도·독도 해양연구기지를 중심으로 체계적이고 과학적인 근거를 바탕으로 독도를 연구하고 기록하고 있답니다.
10월 25일은 '독도의 날'우산도, 삼봉도, 가지도, 석도 등의 이름으로 불렸던 독도1900년 10월 25일 대한제국은 칙령 제41호를 통해울릉도를 울도군으로 울릉도 도감을 울도 군수로 격상 (배계주 울도군 초대 군수)독도 역시 울도군 군수의 관할 하에 두었다.독도의 날(

In [29]:
# 어린이용 기사 인덱스: 84, 유사한 뉴스 기사 인덱스: 5098, 유사도: 0.7504144689210519
print(dropna_kids.iloc[84]['Content'])
print(dropna_bigs.iloc[5098]['Content'])

132억 년 전에 생성된 최고령 블랙홀이 최근 관측되었다. NASA 제공 우주 관측을 시작한 이래(일정한 때로부터 지금까지)로, 가장 나이가 많은 블랙홀이 발견되었어요. ‘검은 구멍(black hole)’이라는 뜻을 가진 ‘블랙홀’은 다른 물체를 끌어당기는 힘인 ‘중력’이 매우 강해 주변에 있는 물체를 모두 빨아들이는 천체를 말해요. 빛조차 빠져나가지 못할 정도로 세게 끌어당겨 천체가 검은색으로 보여서 ‘블랙홀’이라 불리지요. 미국 항공우주국(NASA·나사)은 제임스웹 우주망원경과 찬드라 X선 관측소가 빅뱅(우주의 탄생을 가져왔다고 보는 거대한 폭발) 발생 4억7000만 년 후에 형성된 블랙홀을 발견했다고 최근 밝혔습니다. 나사는 우주의 나이를 약 137억 7000만 년으로 추정하는데, 이번에 발견된 블랙홀은 132억 년 전에 생성된 것으로 예상해요. 이는 지금까지 알려진 블랙홀 중 가장 나이가 많은 것. 나이뿐 아니라 크기와 무게 또한 주목돼요. 우리 은하계에 있는 블랙홀보다 10배가량 더 크고, 무게는 해당 블랙홀이 속한 은하계에 있는 다른 별들과 비슷한 정도에까지 이를 수 있는 것으로 보여요. 보통 블랙홀의 무게는 자신이 속한 은하계에 있는 다른 별들에 비해 매우 적은 수준이지요. 나사는 “우주가 탄생하고 난 이후 매우 이른 시기에 이렇게 거대한 것이 존재했다는 것이 너무나도 놀랍다”고 설명했어요.​
미 항공우주국(NASA) 제공빅뱅 이후 4억 7,000만 년 뒤 만들어진 최고령 블랙홀이 발견됐다.6일(현지 시간) AP 통신에 따르면 미국 항공우주국(NASA) 제임스웹 우주망원경과 찬드라 X선 관측소는 이날 과학 저널 네이처 천문학(Nature Astronomy)에서 이같이 발표했다.우주의 나이가 137억 년 정도인 것을 감안할 때 이 블랙홀은 빅뱅 이후 4억 7,000년 뒤에 만들어졌다. 나이는 관측 이래 가장 오래된 132억 살 정도로 추정된다.이 블랙홀의 크기는 우리은하에 있는 것보다 10배가량 더 크며, 무게는 해당 은하계에 있는 별들의 총질량의 